In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import Libraries**

In [6]:
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)

In [5]:
%%python –version
import tensorflow as tf
from tensorflow.keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import glob

python: can't open file '<unprintable file name>': [Errno 2] No such file or directory


CalledProcessError: Command 'b'import tensorflow as tf\nfrom tensorflow.keras.utils import normalize\nimport os\nimport cv2\nfrom PIL import Image\nimport numpy as np\nfrom matplotlib import pyplot as plt\nfrom sklearn.preprocessing import MinMaxScaler\nfrom tensorflow.keras.optimizers import Adam\nimport glob\n'' returned non-zero exit status 2.

# read pathes of data

In [3]:
image_directory = '/kaggle/input/modis-dataset/images'
mask_directory = '/kaggle/input/modis-dataset/annotations'
test_image_directory = '/kaggle/input/d/abdulrahmanmagdy01/test-images/test'

# read names of images in data

In [4]:
image_names = glob.glob("/kaggle/input/modis-dataset/images/*.jpg")
mask_names = glob.glob("/kaggle/input/modis-dataset/annotations/*.png")
image_test_names = glob.glob("/kaggle/input/d/abdulrahmanmagdy01/test-images/test/*.jpg")

print(image_names[0:6])
print(mask_names[0:6])
print(image_test_names[0:6])

['/kaggle/input/modis-dataset/images/45.jpg', '/kaggle/input/modis-dataset/images/56.jpg', '/kaggle/input/modis-dataset/images/20.jpg', '/kaggle/input/modis-dataset/images/58.jpg', '/kaggle/input/modis-dataset/images/6.jpg', '/kaggle/input/modis-dataset/images/76.jpg']
['/kaggle/input/modis-dataset/annotations/41_GT.png', '/kaggle/input/modis-dataset/annotations/49_GT.png', '/kaggle/input/modis-dataset/annotations/21_GT.png', '/kaggle/input/modis-dataset/annotations/76_GT.png', '/kaggle/input/modis-dataset/annotations/15_GT.png', '/kaggle/input/modis-dataset/annotations/36_GT.png']
['/kaggle/input/d/abdulrahmanmagdy01/test-images/test/208.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/333.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/369.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/89.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/275.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/212.jpg']


# sort names of images in data

In [5]:
image_names = sorted(image_names, key = lambda x: (len (x), x))
mask_names = sorted(mask_names, key = lambda x: (len (x), x))
image_test_names = sorted(image_test_names, key = lambda x: (len (x), x))
print(image_names[0:6])
print(mask_names[0:6])
print(image_test_names[0:6])

['/kaggle/input/modis-dataset/images/0.jpg', '/kaggle/input/modis-dataset/images/1.jpg', '/kaggle/input/modis-dataset/images/2.jpg', '/kaggle/input/modis-dataset/images/3.jpg', '/kaggle/input/modis-dataset/images/4.jpg', '/kaggle/input/modis-dataset/images/5.jpg']
['/kaggle/input/modis-dataset/annotations/0_GT.png', '/kaggle/input/modis-dataset/annotations/1_GT.png', '/kaggle/input/modis-dataset/annotations/2_GT.png', '/kaggle/input/modis-dataset/annotations/3_GT.png', '/kaggle/input/modis-dataset/annotations/4_GT.png', '/kaggle/input/modis-dataset/annotations/5_GT.png']
['/kaggle/input/d/abdulrahmanmagdy01/test-images/test/80.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/81.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/82.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/83.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/84.jpg', '/kaggle/input/d/abdulrahmanmagdy01/test-images/test/85.jpg']


# resize and normalization for data

In [6]:
images = [cv2.imread(img) for img in image_names]
masks = [cv2.imread(mask,0) for mask in mask_names]
for i in range(len(images)):
    images[i] = cv2.resize(images[i] , (256,256))
    images[i] = cv2.normalize(images[i], None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    masks[i] = cv2.resize(masks[i] , (256,256))
    masks[i] = cv2.normalize(masks[i], None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
test_images = [cv2.imread(img) for img in image_test_names]
for j in range(len(test_images)):
    test_images[j] = cv2.resize(test_images[j] , (256,256))
    test_images[j] = cv2.normalize(test_images[j], None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)

In [7]:
print(len(images))
print(len(masks))

80
80


In [ ]:
for i in range(len(images)):
    masks[i].resize(256,256,1)
#     plt.imshow(images[i])
#     plt.pause(0.01)
#     plt.imshow(masks[i])
#     plt.pause(0.01)
    r_to_l_img = tf.image.flip_left_right(images[i])
    r_to_l_mask = tf.image.flip_left_right(masks[i])
#     plt.imshow(r_to_l_img)
#     plt.pause(0.01)
#     plt.imshow(r_to_l_mask)
#     plt.pause(0.01)
    images.append(r_to_l_img)
    masks.append(r_to_l_mask)
#     plt.imshow(images[-1])
#     plt.pause(0.01)
#     plt.imshow(masks[-1])
#     plt.pause(0.01)
    u_to_d_img = tf.image.flip_up_down(images[i])
    u_to_d_mask = tf.image.flip_up_down(masks[i])
#     plt.imshow(u_to_d_img)
#     plt.pause(0.01)
#     plt.imshow(u_to_d_mask)
#     plt.pause(0.01)
    images.append(u_to_d_img)
    masks.append(u_to_d_mask)
#     plt.imshow(images[-1])
#     plt.pause(0.01)
#     plt.imshow(masks[-1])
#     plt.pause(0.01)

# convert data to numpy array

In [ ]:
image_dataset = np.array(images)
mask_dataset = np.array(masks)
test_image_dataset = np.array(test_images)

In [ ]:
j=80
i=0
for i in range(1):    #len(image_dataset)
    path = 'E:\\study\\faculty of computer sience\\Graduation project\\Marwa task\\images\\80.jpg'
    r_to_l_img = tf.image.flip_left_right(image_dataset[i])
    print(type(image_dataset))

In [ ]:
mask_dataset.resize(80,256,256,1)

# print data shapes

In [ ]:
print("Image data shape is: ", image_dataset.shape)
print("Mask data shape is: ", mask_dataset.shape)
print("Test_Image data shape is: ", test_image_dataset.shape)

# visualize image and its mask

In [ ]:
i=78
plt.imshow(images[i])
plt.pause(0.01)
plt.imshow(masks[i] , cmap='gray')

# prepare data for train

In [ ]:
X = image_dataset
Y = mask_dataset

# Build our U-net model

In [ ]:
# Building Unet by dividing encoder and decoder into blocks

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

# read data shapes

In [ ]:
IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
input_shape

# Call model and print his summary

In [ ]:
model = build_unet(input_shape, n_classes=1)
model.compile(optimizer=Adam(learning_rate = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# train model 

In [ ]:
train = model.fit(X, Y, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=200,  
                    shuffle=False)

# Prepare test data

In [ ]:
test = test_image_dataset

# Predict our trained model

In [ ]:
predict_model = model.predict(test)

# Visaulize our prediction

In [ ]:
i=1
plt.imshow(test_images[i])
plt.pause(0.01)
plt.imshow(predict_model[i],cmap="gray")